<a href="https://colab.research.google.com/github/prtkmhn/ATS-Resume-Reviewer/blob/prateek-development/ATS_TALK_SCREENSHOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import time
import subprocess
import requests
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain_community.document_loaders import TextLoader, WebBaseLoader
from langchain.chains import LLMChain
import speech_recognition as sr
import pyttsx3
from langchain.chains.retrieval import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import OpenAIEmbeddings
from pynput import keyboard
import asyncio
from asyncio import WindowsSelectorEventLoopPolicy
import pyautogui
import cv2
import numpy as np
from PIL import Image
import google.generativeai as genai

asyncio.set_event_loop_policy(WindowsSelectorEventLoopPolicy())

load_dotenv()
os.environ['GOOGLE_API_KEY'] = "YOUR API KEY"
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
# Initialize Gemini Pro Vision model


In [ ]:


# Initialize Gemini Pro Vision model
vision_model = genai.GenerativeModel('gemini-pro-vision')
# Initialize Gemini Pro model
gemini_model = genai.GenerativeModel('gemini-pro')

class LanguageModelProcessor:
    def __init__(self, max_tokens=256, temperature=0.7):
        self.llm = ChatGroq(temperature=temperature, model_name="llama3-70b-8192", groq_api_key=os.getenv("GROQ_API_KEY"), max_tokens=max_tokens)
        self.memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

        with open('system_prompt.txt', 'r') as file:
            system_prompt = file.read().strip()

        self.embed_model = "text-embedding-3-small"
        self.embeddings = OpenAIEmbeddings(model=self.embed_model, openai_api_key=os.getenv("OPENAI_API_KEY"))

        self.documents = []

        self.vector_store = None
        self.retriever = None
        self.retrieve_chain = None

        self.prompt = ChatPromptTemplate.from_template('''
            Here is some information extracted from a screenshot:
            <context>
            {context}
            </context>
            Based on this information and the user's resume, provide insights or suggestions that could be beneficial. Please ignore the context if no information is provided
            If the conversation is not about the user's resume but in general about any other topic, please oblige and continue the conversation
            Question: {input}
        ''')

        self.document_chain = create_stuff_documents_chain(self.llm, self.prompt)

        self.conversation_prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(system_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            HumanMessagePromptTemplate.from_template("{text}")
        ])

        self.conversation = LLMChain(
            llm=self.llm,
            prompt=self.conversation_prompt,
            memory=self.memory
        )

    def process(self, text, screenshot_context=None):
        self.memory.chat_memory.add_user_message(text)

        start_time = time.time()
        if screenshot_context:
            # Use the screenshot context for "on my screen" requests
            response = self.llm.predict(context=screenshot_context, input=text)
        elif self.retrieve_chain is not None:
            response = self.retrieve_chain.invoke({"input": text})
        else:
            response = self.conversation.predict(text=text)
        end_time = time.time()

        if isinstance(response, str):
            response = {'answer': response}

        self.memory.chat_memory.add_ai_message(response['answer'])

        elapsed_time = int((end_time - start_time) * 1000)
        print(f"LLM ({elapsed_time}ms): {response['answer']}")
        return response['answer']


class TextToSpeech:
    def __init__(self):
        self.engine = pyttsx3.init()
        self.stop_flag = False

    def speak(self, text):
        self.stop_flag = False
        self.engine.say(text)
        self.engine.runAndWait()
        if self.stop_flag:
            self.engine.stop()

    def stop(self):
        self.stop_flag = True

class SpeechToText:
    def __init__(self):
        self.recognizer = sr.Recognizer()

    def listen(self):
        with sr.Microphone() as source:
            print("Listening...")
            audio = self.recognizer.listen(source)

        try:
            text = self.recognizer.recognize_google(audio)
            print(f"Human: {text}")
            return text
        except sr.UnknownValueError:
            print("Could not understand audio")
        except sr.RequestError as e:
            print(f"Could not request results from Google Speech Recognition service; {e}")

        return ""

class ConversationManager:
    def __init__(self, max_tokens=256, temperature=0.3):
        self.max_tokens = max_tokens
        self.temperature = temperature
        self.llm = LanguageModelProcessor(max_tokens=self.max_tokens, temperature=self.temperature)

    def on_press(self, key):
        if key == keyboard.Key.enter:
            self.tts.stop()
        elif key.char == 'q':  # Change to the desired key for interruption
            self.tts.stop()

    async def take_screenshot(self):
        screenshot = pyautogui.screenshot()
        screenshot = pyautogui.screenshot("Tempimage.png")
        return screenshot

    async def process_screenshot(self, screenshot):
        response1 = vision_model.generate_content(["Please extract the company name, job name, and job description from this image. Please be as detailed in extracting the information adding any supplementary information needed as well", screenshot], stream=True)
        response1.resolve()
        extracted_info = response1.text

        # Add the extracted information to Groq chat model's memory
        self.llm.memory.chat_memory.add_user_message(extracted_info)

        return extracted_info
    async def on_my_screen(self):
        screenshot = await self.take_screenshot()
        processed_screenshot = await self.process_screenshot(screenshot)
        print("Screenshot taken and processed.")
        print(f"Extracted information: {processed_screenshot}")

        print("Please speak your prompt after the beep...")
        prompt = self.stt.listen()  # Use speech-to-text to capture the prompt

        # Use the extracted information with the provided prompt
        llm_response = self.llm.process(prompt, screenshot_context=processed_screenshot)
        self.tts.speak(llm_response)
        return llm_response

    def add_text_file_to_embedding(self, file_path):
        loader = TextLoader(file_path)
        docs = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        self.llm.documents.extend(text_splitter.split_documents(docs))
        self.llm.vector_store = FAISS.from_documents(self.llm.documents, self.llm.embeddings)
        self.llm.retriever = self.llm.vector_store.as_retriever()
        self.llm.retrieve_chain = create_retrieval_chain(self.llm.retriever, self.llm.document_chain)

    def add_url_to_embedding(self, url):
        try:
            loader = WebBaseLoader(url)
            docs = loader.load()
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
            self.llm.documents.extend(text_splitter.split_documents(docs))
            self.llm.vector_store = FAISS.from_documents(self.llm.documents, self.llm.embeddings)
            self.llm.retriever = self.llm.vector_store.as_retriever()
            self.llm.retrieve_chain = create_retrieval_chain(self.llm.retriever, self.llm.document_chain)
            print(f"Successfully added data from {url} to embeddings.")
        except Exception as e:
            print(f"Error loading URL: {url}")
            print(f"Error message: {str(e)}")

    async def main(self):
        self.tts = TextToSpeech()
        self.stt = SpeechToText()

        listener = keyboard.Listener(on_press=self.on_press)
        listener.start()

        # Add resume.txt to embeddings
        self.add_text_file_to_embedding("resume.txt")

        # Load LinkedIn and GitHub profiles from file if available
        if os.path.exists("profiles.txt"):
            with open("profiles.txt", "r") as file:
                linkedin_url = file.readline().strip()
                github_url = file.readline().strip()
        else:
            linkedin_url = input("Enter your LinkedIn profile URL: ")
            github_url = input("Enter your GitHub profile URL: ")
            with open("profiles.txt", "w") as file:
                file.write(f"{linkedin_url}\n{github_url}")

        # Add LinkedIn profile to embeddings
        self.add_url_to_embedding(linkedin_url)

        # Add GitHub profile to embeddings
        self.add_url_to_embedding(github_url)

        while True:
            input("Press Enter to start listening...")
            transcription_response = self.stt.listen()

            if "goodbye" in transcription_response.lower():
                break

            if "screenshot" in transcription_response.lower():
                screenshot = await self.take_screenshot()
                processed_screenshot = await self.process_screenshot(screenshot)
                print("Screenshot taken and processed.")
                print(f"Extracted information: {processed_screenshot}")

                # Prompt user for confirmation to update resume
                update_resume = input("Do you want to update your resume with the extracted job details? (y/n): ")
                if update_resume.lower() == 'y':
                    self.temp_resume_file.write(f"\n\nNew Job Details:\n{processed_screenshot}")
                    self.temp_resume_file.flush()  # Ensure data is written to the file
                    print("Temporary resume updated with the new job details.")
                transcription_response = self.stt.listen()
            if "textfile" in transcription_response.lower():
                file_path = input("Enter the path to the text file: ")
                self.add_text_file_to_embedding(file_path)
                print(f"Text file '{file_path}' added to the embedding.")
                continue
            if "on my screen" in transcription_response.lower():
                response = await self.on_my_screen()  # Now uses speech for the prompt
                print(f"Response based on screenshot: {response}")

            if transcription_response.strip():
                llm_response = self.llm.process(transcription_response)
                self.tts.speak(llm_response)

if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser(description='Talk to your LLM')
    parser.add_argument('--max_tokens', type=int, default=70, help='Maximum number of tokens for LLM output')
    parser.add_argument('--temperature', type=float, default=0.3, help='Temperature for LLM')
    args = parser.parse_args()

    manager = ConversationManager(max_tokens=args.max_tokens, temperature=args.temperature)
    asyncio.run(manager.main())
